# Training of neural networks

## Training of regular U-Net for sarcomere detection

## Training of time-consistent Siam-U-Net for sarcomere detection

## Training of neural network for analysis of contraction periods